In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np

# Load the unioned GeoJSON
gdf = gpd.read_file(
    r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\LivingstonParish_Parcels_AquisitionProperty_SFHA_Union.geojson'
)
gdf


,OBJECTID,FID from Parcels Aquisition Property Join,Count of Parcels Aquisition Property Join,Match Distance,TARGET_FID,Parcel Number,Global ID,Lot Number,OBJECTID (DEPRECATED),Owner Name,...,AR SUBTRV,BFE Revert,Depth Revert,Dual Zone,Source Citation,GFID,Shape_Length,Shape_Area,SFHA Area (Acres),geometry
0,1,2,0,-1,2,0249110,{8F6B3334-458A-418E-9B6B-000374609775},,1365051,"DICKERSON, JEREMY A & SHERRY T",...,,0,0,,,,466.319721,12910.919354,0.296394,"POLYGON ((3507076.811 658962.989, 3507035.408 ..."
1,2,3,0,-1,3,0465849,{30E2F9D2-FD94-46D2-A4D2-00049623798F},,1389050,"MCQUAIN, MCARTHUR M & NAOMI A",...,,0,0,,,,516.207857,14476.384233,0.332332,"POLYGON ((3406525.143 760443.853, 3406383.601 ..."
2,3,5,0,-1,5,0302349,{63CCEB32-CF46-461E-A849-0007A3C6B4BD},,1413772,"SMITH, CHARLEY R",...,,0,0,,,,323.988739,5037.909946,0.115654,"POLYGON ((3465808.593 728286.062, 3465808.572 ..."
3,4,6,0,-1,6,0347252,{1E3FBD00-D10B-45C4-81BD-0007B25D2618},,1377103,"WAX, BRADY",...,,0,0,,,,333.179932,5828.955136,0.133814,"POLYGON ((3400258.16 720520.829, 3400258.16 72..."
4,5,7,0,-1,7,0615476,{D7F969C6-6952-48A1-A83F-0008EE8E16AF},,1394936,"NEWGARD, JACOB & ZOE W",...,,0,0,,,,419.999816,9799.983764,0.224977,"POLYGON ((3464907.755 711921.429, 3464907.755 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102894,102895,72936,0,-1,72936,0141535,{4D57A237-1095-4620-872F-DFECE05E95BD},,0,,...,None,-9999,-9999,None,22063C_STUDY2,20140410,191.322341,3.839553,0.000088,"POLYGON ((3491618.185 727622.972, 3491561.407 ..."
102895,102896,80414,0,-1,80414,0141556,{FD6F2CD1-93ED-4368-8D41-F7B4877151DE},,1356880,"MCMANUS, TRENTEN",...,None,-9999,-9999,None,22063C_STUDY2,20140410,191.322341,3.839553,0.000088,"POLYGON ((3491618.185 727622.972, 3491561.407 ..."
102896,102897,72936,0,-1,72936,0141535,{4D57A237-1095-4620-872F-DFECE05E95BD},,0,,...,None,-9999,-9999,None,22063C_STUDY2,20140410,38.401446,0.702842,0.000016,"POLYGON ((3491561.407 727699.917, 3491549.959 ..."
102897,102898,77514,0,-1,77514,0141556,{1AD84569-61ED-4530-9C4C-EE8A15C32461},,1356880,"MCMANUS, TRENTEN",...,None,-9999,-9999,None,22063C_STUDY2,20140410,38.401446,0.702842,0.000016,"POLYGON ((3491561.407 727699.917, 3491549.959 ..."


In [10]:
# Ensure area field is numeric
gdf['SFHA Area (Acres)'] = pd.to_numeric(gdf['SFHA Area (Acres)'], errors='coerce').fillna(0)
gdf['Parcel Area (Acres)'] = pd.to_numeric(gdf['Parcel Area (Acres)'], errors='coerce').fillna(0)

'''# Group by Parcel, Flood Zone, and Zone Subtype
grouped = gdf.groupby(['Parcel Number', 'Flood Zone', 'Zone Subtype'])['SFHA Area (Acres)'].sum().reset_index()'''

# Handle missing or blank flood zones
gdf['Flood Zone'] = gdf['Flood Zone'].fillna('').replace('', 'Not in Flood Zone')

# Create a composite ID
gdf['Parcel_ID'] = gdf['OBJECTID'].astype(str) + ' | ' + gdf['Parcel Number'].fillna('')

# Flag missing or placeholder Parcel Numbers
gdf['Missing Parcel Number'] = gdf['Parcel Number'].isna() | (gdf['Parcel Number'].str.strip() == '???')

# Group by Parcel_ID, Flood Zone, and Zone Subtype
grouped = gdf.groupby(['Parcel_ID', 'Flood Zone', 'Zone Subtype']).agg({
    'SFHA Area (Acres)': 'sum',
    'Parcel Area (Acres)': 'first',
    'Parcel Number': 'first',
    'Missing Parcel Number': 'first'
}).reset_index()
grouped

,Parcel_ID,Flood Zone,Zone Subtype,SFHA Area (Acres),Parcel Area (Acres),Parcel Number,Missing Parcel Number
0,1 | 0249110,Not in Flood Zone,,0.296394,0.545072,0249110,False
1,10 | 0067041,Not in Flood Zone,,2.184192,2.184192,0067041,False
2,100 | 0025320,Not in Flood Zone,,0.582782,0.582782,0025320,False
3,1000 | 0470294,Not in Flood Zone,,0.318496,0.318496,0470294,False
4,10000 | 0120055,Not in Flood Zone,,2.583122,3.144196,0120055,False
...,...,...,...,...,...,...,...
48077,9995 | 0608372,Not in Flood Zone,,1.170226,1.170226,0608372,False
48078,9996 | 0582668A,Not in Flood Zone,,0.698496,0.698686,0582668A,False
48079,9997 | 0037309,Not in Flood Zone,,0.728446,2.471566,0037309,False
48080,9998 | 0120931B,Not in Flood Zone,,0.068869,0.068869,0120931B,False


In [24]:
# Normalize fields
gdf['Flood Zone'] = gdf['Flood Zone'].fillna('').str.strip().str.upper()
gdf['SFHA Area (Acres)'] = pd.to_numeric(gdf['SFHA Area (Acres)'], errors='coerce').fillna(0)
gdf['Parcel Area (Acres)'] = pd.to_numeric(gdf['Parcel Area (Acres)'], errors='coerce').fillna(0)

# Treat missing or empty flood zones as "NOT IN FLOOD ZONE"
gdf.loc[gdf['Flood Zone'] == '', 'Flood Zone'] = 'NOT IN FLOOD ZONE'

# Set SFHA Area to 0 where NOT IN FLOOD ZONE
gdf.loc[gdf['Flood Zone'] == 'NOT IN FLOOD ZONE', 'SFHA Area (Acres)'] = 0

# Create composite Parcel ID
gdf['Parcel_ID'] = gdf['OBJECTID'].astype(str) + ' | ' + gdf['Parcel Number'].fillna('')

# Flag missing parcel numbers
gdf['Missing Parcel Number'] = gdf['Parcel Number'].isna() | (gdf['Parcel Number'].str.strip() == '???')

# Base parcel-level data
parcel_df = gdf[['Parcel_ID', 'Parcel Number', 'Parcel Area (Acres)', 'Missing Parcel Number']].drop_duplicates()

# Only include actual flood zones (exclude NOT IN FLOOD ZONE)
flood_gdf = gdf[gdf['Flood Zone'] != 'NOT IN FLOOD ZONE'].copy()

# Group by Parcel_ID and Flood Zone to get area
zone_acres = flood_gdf.groupby(['Parcel_ID', 'Flood Zone'])['SFHA Area (Acres)'].sum().unstack(fill_value=0)
zone_acres.columns = [f'Acres {zone}' for zone in zone_acres.columns]

# Calculate percentage per flood zone
zone_percents = zone_acres.div(parcel_df.set_index('Parcel_ID')['Parcel Area (Acres)'], axis=0) * 100
zone_percents.columns = [col.replace('Acres', 'Percent') for col in zone_acres.columns]

# Merge into final table
final_df = parcel_df.set_index('Parcel_ID').join([zone_acres, zone_percents])
final_df = final_df.reset_index()

# Calculate total SFHA Area and % across all flood zones (excluding NOT IN)
sfha_acre_cols = [col for col in final_df.columns if col.startswith('Acres ')]
final_df['Total SFHA Area'] = final_df[sfha_acre_cols].sum(axis=1)
final_df['Cumulative SFHA %'] = (final_df['Total SFHA Area'] / final_df['Parcel Area (Acres)']) * 100
final_df

,Parcel_ID,Parcel Number,Parcel Area (Acres),Missing Parcel Number,Acres A,Acres AE,Acres VE,Percent A,Percent AE,Percent VE,Total SFHA Area,Cumulative SFHA %
0,1 | 0249110,0249110,0.545072,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1,2 | 0465849,0465849,0.332332,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2,3 | 0302349,0302349,0.115654,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
3,4 | 0347252,0347252,0.133814,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
4,5 | 0615476,0615476,0.224977,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
102894,102895 | 0141535,0141535,2.260660,False,0.000088,0.0,0.0,0.003899,0.0,0.0,0.000088,0.003899
102895,102896 | 0141556,0141556,0.999639,False,0.000088,0.0,0.0,0.008818,0.0,0.0,0.000088,0.008818
102896,102897 | 0141535,0141535,2.260660,False,0.000016,0.0,0.0,0.000714,0.0,0.0,0.000016,0.000714
102897,102898 | 0141556,0141556,0.215676,False,0.000016,0.0,0.0,0.007481,0.0,0.0,0.000016,0.007481


In [11]:
# Calculate % per flood zone per parcel
grouped['Percent of Parcel'] = (grouped['SFHA Area (Acres)'] / grouped['Parcel Area (Acres)']) * 100

# Calculate cumulative SFHA % per parcel
sfha_totals = grouped.groupby('Parcel_ID')['SFHA Area (Acres)'].sum().rename('Total SFHA Area')
grouped = grouped.merge(sfha_totals, on='Parcel_ID')
grouped['Cumulative SFHA %'] = (grouped['Total SFHA Area'] / grouped['Parcel Area (Acres)']) * 100

# Sort for readability
grouped = grouped.sort_values(by=['Parcel_ID', 'Percent of Parcel'], ascending=[True, False])
grouped


,Parcel_ID,Flood Zone,Zone Subtype,SFHA Area (Acres),Parcel Area (Acres),Parcel Number,Missing Parcel Number,Percent of Parcel,Total SFHA Area,Cumulative SFHA %
0,1 | 0249110,Not in Flood Zone,,0.296394,0.545072,0249110,False,54.377001,0.296394,54.377001
1,10 | 0067041,Not in Flood Zone,,2.184192,2.184192,0067041,False,100.000000,2.184192,100.000000
2,100 | 0025320,Not in Flood Zone,,0.582782,0.582782,0025320,False,100.000000,0.582782,100.000000
3,1000 | 0470294,Not in Flood Zone,,0.318496,0.318496,0470294,False,100.000000,0.318496,100.000000
4,10000 | 0120055,Not in Flood Zone,,2.583122,3.144196,0120055,False,82.155254,2.583122,82.155254
...,...,...,...,...,...,...,...,...,...,...
48077,9995 | 0608372,Not in Flood Zone,,1.170226,1.170226,0608372,False,100.000000,1.170226,100.000000
48078,9996 | 0582668A,Not in Flood Zone,,0.698496,0.698686,0582668A,False,99.972759,0.698496,99.972759
48079,9997 | 0037309,Not in Flood Zone,,0.728446,2.471566,0037309,False,29.473064,0.728446,29.473064
48080,9998 | 0120931B,Not in Flood Zone,,0.068869,0.068869,0120931B,False,100.000000,0.068869,100.000000


In [26]:
final_df['OID'] = final_df['Parcel_ID'].str.split(' | ').str[0]

# Ensure all Acres and Percent fields are numeric
for col in final_df.columns:
    if col.startswith('Acres ') or col.startswith('Percent ') or col in ['Total SFHA Area', 'Cumulative SFHA %']:
        final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0).round(4)  # round to 4 decimal places
final_df

,Parcel_ID,Parcel Number,Parcel Area (Acres),Missing Parcel Number,Acres A,Acres AE,Acres VE,Percent A,Percent AE,Percent VE,Total SFHA Area,Cumulative SFHA %,OID
0,1 | 0249110,0249110,0.545072,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,1
1,2 | 0465849,0465849,0.332332,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,2
2,3 | 0302349,0302349,0.115654,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,3
3,4 | 0347252,0347252,0.133814,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,4
4,5 | 0615476,0615476,0.224977,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102894,102895 | 0141535,0141535,2.260660,False,0.0001,0.0,0.0,0.0039,0.0,0.0,0.0001,0.0039,102895
102895,102896 | 0141556,0141556,0.999639,False,0.0001,0.0,0.0,0.0088,0.0,0.0,0.0001,0.0088,102896
102896,102897 | 0141535,0141535,2.260660,False,0.0000,0.0,0.0,0.0007,0.0,0.0,0.0000,0.0007,102897
102897,102898 | 0141556,0141556,0.215676,False,0.0000,0.0,0.0,0.0075,0.0,0.0,0.0000,0.0075,102898


In [27]:
final_df.dtypes

Parcel_ID                 object
Parcel Number             object
Parcel Area (Acres)      float64
Missing Parcel Number       bool
Acres A                  float64
Acres AE                 float64
Acres VE                 float64
Percent A                float64
Percent AE               float64
Percent VE               float64
Total SFHA Area          float64
Cumulative SFHA %        float64
OID                       object
dtype: object

In [ ]:
# Replace inf/-inf with NaN
final_df.replace([np.inf, -np.inf], np.nan, inplace=True)
final_df

In [ ]:
gdf_summary = gdf[['Parcel_ID', 'geometry']].drop_duplicates()
gdf_joined = gdf_summary.merge(final_df, on='Parcel_ID')
gdf_joined

,Parcel_ID,geometry,Parcel Number,Parcel Area (Acres),Missing Parcel Number,Acres A,Acres AE,Acres VE,Percent A,Percent AE,Percent VE,Total SFHA Area,Cumulative SFHA %,OID
0,1 | 0249110,"POLYGON ((3507076.811 658962.989, 3507035.408 ...",0249110,0.545072,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,1
1,2 | 0465849,"POLYGON ((3406525.143 760443.853, 3406383.601 ...",0465849,0.332332,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,2
2,3 | 0302349,"POLYGON ((3465808.593 728286.062, 3465808.572 ...",0302349,0.115654,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,3
3,4 | 0347252,"POLYGON ((3400258.16 720520.829, 3400258.16 72...",0347252,0.133814,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,4
4,5 | 0615476,"POLYGON ((3464907.755 711921.429, 3464907.755 ...",0615476,0.224977,False,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102894,102895 | 0141535,"POLYGON ((3491618.185 727622.972, 3491561.407 ...",0141535,2.260660,False,0.0001,0.0,0.0,0.0039,0.0,0.0,0.0001,0.0039,102895
102895,102896 | 0141556,"POLYGON ((3491618.185 727622.972, 3491561.407 ...",0141556,0.999639,False,0.0001,0.0,0.0,0.0088,0.0,0.0,0.0001,0.0088,102896
102896,102897 | 0141535,"POLYGON ((3491561.407 727699.917, 3491549.959 ...",0141535,2.260660,False,0.0000,0.0,0.0,0.0007,0.0,0.0,0.0000,0.0007,102897
102897,102898 | 0141556,"POLYGON ((3491561.407 727699.917, 3491549.959 ...",0141556,0.215676,False,0.0000,0.0,0.0,0.0075,0.0,0.0,0.0000,0.0075,102898


In [20]:
gdf_joined.dtypes

Parcel_ID                  object
geometry                 geometry
Parcel Number              object
Parcel Area (Acres)       float64
Missing Parcel Number        bool
Acres A                   float64
Acres AE                  float64
Acres VE                  float64
Percent A                 float64
Percent AE                float64
Percent VE                float64
Total SFHA Area           float64
Cumulative SFHA %         float64
OID                        object
dtype: object

In [21]:
# Then write:
gdf_joined.to_file(r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\parcel_sfha_summary.gpkg', layer='sfha_summary', driver='GPKG')

In [28]:
# Export to CSV for review
final_df.to_csv(r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\parcel_flood_zone_summary.csv', index=False)


In [ ]:
final_df.to_file(r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\parcel_flood_zone_summary.shp', driver='ESRI Shapefile')